# 스팸탐지

-> nlp 프로젝트의 스타일을 익혀보자

순서 : 문자 > 토큰화 > 인코딩(word2idx) > 패딩 > 라벨 원핫 인코딩 > 임베딩  > 데이터셋 객체를 생성하여 fit

In [ ]:
import argparse
import gensim.downloader as api
import numpy as np
import os
import shutil
import tensorflow as tf

from sklearn.metrics import accuracy_score, confusion_matrix # 딥러닝을 할 때 정확도와 혼돈행렬은 사이킷런 라이브러리를 사용

In [ ]:
DATA_DIR = "data"
EMBEDDING_NUMPY_FILE = os.path.join(DATA_DIR, "E.npy")
EMBEDDING_MODEL = "glove-wiki-gigaword-300" # gensim에 임베딩 모델을 지정하면 원하는 임베딩 모델을 가져다 준다.(ex w2v, glove)
EMBEDDING_DIM = 300
NUM_CLASSES = 2
BATCH_SIZE = 128
NUM_EPOCHS = 3

# data distribution is 4827 ham and 747 spam (total 5574), which 
# works out to approx 87% ham and 13% spam, so we take reciprocals
# and this works out to being each spam (1) item as being approximately
# 8 times as important as each ham (0) message.
CLASS_WEIGHTS = { 0: 1, 1: 8 }

tf.random.set_seed(42)

```
데이터셋에는 5500개의 sms레코드가 있고 그중 700개가 스팸, 4800개가 햄이다 레이블은 0이 햄, 1이 스팸이다. 
```

In [ ]:
# UCI SMS 스팸 수집 데이터셋으로 아래 코드는 파일을 다운로드하고 파싱해서 SMS 메시지 목록과 해당 레이블을 생성한다. 
def download_and_read(url):
    local_file = url.split('/')[-1]
    p = tf.keras.utils.get_file(local_file, url, 
        extract=True, cache_dir=".")
    labels, texts = [], []
    local_file = os.path.join("datasets", "SMSSpamCollection")
    with open(local_file, "r") as fin:
        for line in fin:
            label, text = line.strip().split('\t')
            labels.append(1 if label == "spam" else 0)
            texts.append(text)
    return texts, labels


DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"
texts, labels = download_and_read(DATASET_URL)

203415/203415 [==============================] - 0s 2us/step


In [ ]:
texts

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though",
 "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",
 'Even my brother is not like to speak with me. They treat me like aids patent.',
 "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune",
 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.',
 'Had you

In [ ]:
labels

[0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


```
데이터를 사용 준비

텍스트는 정수로 신경망에 공급돼야하고, 여기서는 케라스 토크나이저를 사용한다. 
sms 텍스트를 단어 시퀀스로 변환한 후 토크나이저의 fit_on_texts로 어휘를 생성한다.

마지막으로 망은 고정 길이이므로 pad sequences로 짧은 애들은 0으로 채운다.
예외적으로 긴 애들은 보통 maxlen으로 제한한다. 이 경우 maxlen보다 더 긴 애들은 잘려 나가고 더 짧은 토큰들은 채운다. 
```

In [ ]:
# tokenize and pad text
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(texts)
tokenizer # 토큰화

In [ ]:
# tokenize and pad text
text_sequences = tokenizer.texts_to_sequences(texts)
text_sequences # 결과를 보니 모든 문장의 모든 텍스트들이 토크나이저로 짤리고 texts_to로 정수가 된다. 시퀀스 데이터로 만든다는 것이 정수인가보다.

[[49,
  471,
  4435,
  842,
  755,
  658,
  64,
  8,
  1327,
  88,
  123,
  351,
  1328,
  148,
  2996,
  1329,
  67,
  58,
  4436,
  144],
 [46, 336, 1499, 472, 6, 1940],
 [47,
  489,
  8,
  19,
  4,
  797,
  901,
  2,
  176,
  1941,
  1105,
  659,
  1942,
  2331,
  261,
  2332,
  71,
  1941,
  2,
  1943,
  2,
  337,
  489,
  555,
  960,
  73,
  391,
  174,
  660,
  392,
  2997],
 [6, 248, 150, 23, 382, 2998, 6, 139, 154, 57, 150],
 [1024, 1, 98, 108, 69, 490, 2, 961, 69, 1944, 221, 112, 473],
 [798,
  129,
  67,
  1690,
  145,
  109,
  158,
  1945,
  21,
  7,
  38,
  338,
  89,
  902,
  55,
  116,
  414,
  3,
  44,
  12,
  14,
  86,
  1946,
  46,
  365,
  960,
  4437,
  2,
  68,
  323,
  232,
  2,
  2999],
 [210, 11, 633, 9, 25, 55, 2, 383, 36, 10, 110, 718, 10, 55, 4438, 4439],
 [72,
  235,
  13,
  1204,
  2333,
  2334,
  1947,
  2335,
  2336,
  2337,
  799,
  118,
  109,
  609,
  72,
  13,
  1025,
  12,
  51,
  1691,
  843,
  393,
  2,
  1106,
  13,
  249,
  1025],
 [719,
  72,
  4

In [ ]:
text_sequences = tf.keras.preprocessing.sequence.pad_sequences(text_sequences)
text_sequences

array([[   0,    0,    0, ...,   58, 4436,  144],
       [   0,    0,    0, ...,  472,    6, 1940],
       [   0,    0,    0, ...,  660,  392, 2997],
       ...,
       [   0,    0,    0, ...,  107,  251, 9008],
       [   0,    0,    0, ...,  200,   12,   47],
       [   0,    0,    0, ...,    2,   61,  268]], dtype=int32)

In [ ]:
# ↓둘 다 20개이다 = 즉, 정수로 변한 건데 pad가 앞으로 채워지나보다.
texts[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [ ]:
text_sequences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [ ]:
num_records = len(text_sequences)
max_seqlen = len(text_sequences[0])
print("{:d} sentences, max length: {:d}".format(num_records, max_seqlen))

5574 sentences, max length: 189


```
레이블은 범주형 혹은 원핫 인코딩으로 변환한다.
```

In [ ]:
# labels
# 인자로 라벨과 클래스 개수를 준다. -> 이런 걸 보면 for문을 쓰지 않아도 알아서 labels 전체가 들어간다.
# 즉 나도 labels처럼 넘파이 형태로 만들기만 하면 내 데이터로 학습할 수 있다는 것이다.
cat_labels = tf.keras.utils.to_categorical(labels, num_classes=NUM_CLASSES) 

```
토크나이저는 기본으로 word2idx를 지원한다(엑세스 할 수 있게 해준다) 이는 기본적으로 어휘의 idx 위치에 대한 사전이다.

word -> idx
idx -> word를 제공
```

In [ ]:
# vocabulary
word2idx = tokenizer.word_index # 빈도수가 높은 단어들이 먼저 오는 순서대로 숫자와 매칭되어 있다.
word2idx # 이것으로 텍스트 > 정수를 매칭할 수 있겠다.

{'i': 1,
 'to': 2,
 'you': 3,
 'a': 4,
 'the': 5,
 'u': 6,
 'and': 7,
 'in': 8,
 'is': 9,
 'me': 10,
 'my': 11,
 'for': 12,
 'your': 13,
 'it': 14,
 'of': 15,
 'call': 16,
 'have': 17,
 'on': 18,
 '2': 19,
 'that': 20,
 'now': 21,
 'are': 22,
 'so': 23,
 'but': 24,
 'not': 25,
 'or': 26,
 'do': 27,
 'can': 28,
 'at': 29,
 "i'm": 30,
 'ur': 31,
 'get': 32,
 'will': 33,
 'if': 34,
 'be': 35,
 'with': 36,
 'just': 37,
 'no': 38,
 'we': 39,
 'this': 40,
 '4': 41,
 'gt': 42,
 'lt': 43,
 'up': 44,
 'when': 45,
 'ok': 46,
 'free': 47,
 'from': 48,
 'go': 49,
 'how': 50,
 'all': 51,
 'out': 52,
 'what': 53,
 'know': 54,
 'like': 55,
 'good': 56,
 'then': 57,
 'got': 58,
 'come': 59,
 'was': 60,
 'its': 61,
 'am': 62,
 'time': 63,
 'only': 64,
 'day': 65,
 'love': 66,
 'there': 67,
 'send': 68,
 'he': 69,
 'want': 70,
 'text': 71,
 'as': 72,
 'txt': 73,
 'one': 74,
 'going': 75,
 'by': 76,
 'ü': 77,
 "i'll": 78,
 'need': 79,
 'home': 80,
 'about': 81,
 'r': 82,
 'lor': 83,
 'sorry': 84,
 'stop'

In [ ]:
word2idx.keys() #key 애는 진짜로 단어들이 있고

dict_keys(['i', 'to', 'you', 'a', 'the', 'u', 'and', 'in', 'is', 'me', 'my', 'for', 'your', 'it', 'of', 'call', 'have', 'on', '2', 'that', 'now', 'are', 'so', 'but', 'not', 'or', 'do', 'can', 'at', "i'm", 'ur', 'get', 'will', 'if', 'be', 'with', 'just', 'no', 'we', 'this', '4', 'gt', 'lt', 'up', 'when', 'ok', 'free', 'from', 'go', 'how', 'all', 'out', 'what', 'know', 'like', 'good', 'then', 'got', 'come', 'was', 'its', 'am', 'time', 'only', 'day', 'love', 'there', 'send', 'he', 'want', 'text', 'as', 'txt', 'one', 'going', 'by', 'ü', "i'll", 'need', 'home', 'about', 'r', 'lor', 'sorry', 'stop', 'still', 'see', 'n', 'back', 'today', 'da', 'our', 'dont', 'reply', 'k', 'she', 'mobile', "don't", 'take', 'hi', 'tell', 'new', 'please', 'later', 'her', 'pls', 'any', 'think', 'been', 'they', 'phone', 'here', 'week', 'did', 'dear', 'some', 'well', 'has', '1', 'night', 'much', 'd', 'great', 'oh', 'who', 'hope', 'claim', 'an', 'hey', 'msg', 'where', 'him', 'more', 'too', 'happy', 'had', 'yes', 'gi

In [ ]:
word2idx.values()

dict_values([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 

In [ ]:
idx2word = {v:k for k, v in word2idx.items()}
idx2word # 반대는 for문을 사용하여 직접 만든다.

{1: 'i',
 2: 'to',
 3: 'you',
 4: 'a',
 5: 'the',
 6: 'u',
 7: 'and',
 8: 'in',
 9: 'is',
 10: 'me',
 11: 'my',
 12: 'for',
 13: 'your',
 14: 'it',
 15: 'of',
 16: 'call',
 17: 'have',
 18: 'on',
 19: '2',
 20: 'that',
 21: 'now',
 22: 'are',
 23: 'so',
 24: 'but',
 25: 'not',
 26: 'or',
 27: 'do',
 28: 'can',
 29: 'at',
 30: "i'm",
 31: 'ur',
 32: 'get',
 33: 'will',
 34: 'if',
 35: 'be',
 36: 'with',
 37: 'just',
 38: 'no',
 39: 'we',
 40: 'this',
 41: '4',
 42: 'gt',
 43: 'lt',
 44: 'up',
 45: 'when',
 46: 'ok',
 47: 'free',
 48: 'from',
 49: 'go',
 50: 'how',
 51: 'all',
 52: 'out',
 53: 'what',
 54: 'know',
 55: 'like',
 56: 'good',
 57: 'then',
 58: 'got',
 59: 'come',
 60: 'was',
 61: 'its',
 62: 'am',
 63: 'time',
 64: 'only',
 65: 'day',
 66: 'love',
 67: 'there',
 68: 'send',
 69: 'he',
 70: 'want',
 71: 'text',
 72: 'as',
 73: 'txt',
 74: 'one',
 75: 'going',
 76: 'by',
 77: 'ü',
 78: "i'll",
 79: 'need',
 80: 'home',
 81: 'about',
 82: 'r',
 83: 'lor',
 84: 'sorry',
 85: 's

In [ ]:
len(idx2word)

9009

In [ ]:
word2idx["PAD"] = 0 # 정수(seq)가 1번부터 매칭되므로 0은 PAD로 넣음
idx2word[0] = "PAD"
vocab_size = len(word2idx)
print("vocab size: {:d}".format(vocab_size))

vocab size: 9010


```
데이터셋 객체 만듬
```

In [ ]:
# ★★★ 매우 중요한 것으로 tf의 data api를 사용하면 데이터를 쉽게 사용할 수 있다.
dataset = tf.data.Dataset.from_tensor_slices((text_sequences, cat_labels))
dataset = dataset.shuffle(10000)

'''
교과목 과제 2에서도 이와 동일한 방식으로 한다.
training_data = tf.data.Dataset.from_tensor_slices((tr_img, tr_class))
validation_data = tf.data.Dataset.from_tensor_slices((te_img, te_class))
'''

test_size = num_records // 4
val_size = (num_records - test_size) // 10

test_dataset = dataset.take(test_size)
val_dataset = dataset.skip(test_size).take(val_size)
train_dataset = dataset.skip(test_size + val_size)

test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

```
임베딩 행렬 구축
word2idx를 가져와서 E(임베딩 행렬)에 idx행에 word의 임베딩 결과를 넣음

-> 따라서 임베딩 행렬 shape 결과는 단어개수만큼 idx행이 생기니 9010 x 300이다.
```

In [ ]:
def build_embedding_matrix(sequences, word2idx, embedding_dim, 
        embedding_file):
    if os.path.exists(embedding_file):
        E = np.load(embedding_file)
    else:
        vocab_size = len(word2idx)
        E = np.zeros((vocab_size, embedding_dim))
        word_vectors = api.load(EMBEDDING_MODEL) # glove 임베딩 모델 가져옴
        for word, idx in word2idx.items():
            try:
                E[idx] = word_vectors.word_vec(word)
            except KeyError:   # word not in embedding
                pass
            # except IndexError: # UNKs are mapped to seq over VOCAB_SIZE as well as 1
            #     pass
        np.save(embedding_file, E)
    return E 

```
스팸 분류기 정의

1. 신경망을 무작위로 처음 훈련 시키면 scratch o 가중치 훈련 True

2. 전이 학습은 임베딩 행렬에서 가중치를 설정하지만 가중치는 False

3. 미세 조정은 임베딩 행렬에서 가중치를 설정하고 가중치도 학습 True

-> 2번 방식은 freezing 방식으로 사전 학습시킨 임베딩 레이어는 가중치 갱신을 하지 않고 뒤에 분류기만 학습한다.

3번 방식은 fine tunning 방식으로 사전 학습시킨 임베딩 레이어와 분류기 둘 다 가중치를 갱신한다. 

따라서 임베딩 레이어는 이미 가중치 갱신이 되어있기 때문에 추가로 가중치 갱신이 일어나는 것이다.
```

In [ ]:
# 정수 시퀀스가 입력으로 들어옴
class SpamClassifierModel(tf.keras.Model):
    def __init__(self, vocab_sz, embed_sz, input_length,
            num_filters, kernel_sz, output_sz, 
            run_mode, embedding_weights, 
            **kwargs):
        super(SpamClassifierModel, self).__init__(**kwargs)

        ''' 
        훈련 모드에 따라서 임베딩을 처음부터 학습시키는지, 전이학습시키는지, 미세 조정을 하는 지에 따라
        임베딩 계층이 약간 달라진다. + wow.. 다 트레이너블 할 수 있네
        '''

        if run_mode == "scratch":
            self.embedding = tf.keras.layers.Embedding(vocab_sz, # 그 text_seq가 입력으로 들어옴
                embed_sz, # 입력 seq를 이 사이즈의 벡터로 변환됨
                input_length=input_length,
                trainable=True)
        elif run_mode == "vectorizer":
            self.embedding = tf.keras.layers.Embedding(vocab_sz, 
                embed_sz,
                input_length=input_length,
                weights=[embedding_weights], # 가중치를 사전 학습시킨 임베딩 가중치로 적용
                trainable=False)
        else:
            self.embedding = tf.keras.layers.Embedding(vocab_sz, 
                embed_sz,
                input_length=input_length,
                weights=[embedding_weights],
                trainable=True)
            
        self.dropout = tf.keras.layers.SpatialDropout1D(0.2)
        self.conv = tf.keras.layers.Conv1D(filters=num_filters,
            kernel_size=kernel_sz,
            activation="relu")
        self.pool = tf.keras.layers.GlobalMaxPooling1D()
        self.dense = tf.keras.layers.Dense(output_sz, # num_classes = 2
            activation="softmax"
        )

    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.conv(x)
        x = self.pool(x)
        x = self.dense(x)
        return x

In [ ]:
# embedding(단어를 정수로 바꾼 seq, word2idx, 300, 사전 학습 임베딩 가중치 파일.npy
E = build_embedding_matrix(text_sequences, word2idx, EMBEDDING_DIM,
    EMBEDDING_NUMPY_FILE)
print("Embedding matrix:", E.shape)


# model definition
conv_num_filters = 256
conv_kernel_size = 3
model = SpamClassifierModel(
    vocab_size, EMBEDDING_DIM, max_seqlen, 
    conv_num_filters, conv_kernel_size, NUM_CLASSES,
    "scratch", E)
model.build(input_shape=(None, max_seqlen))
model.summary()

# compile and train
model.compile(optimizer="adam", loss="categorical_crossentropy",
    metrics=["accuracy"])

# train model
model.fit(train_dataset, epochs=NUM_EPOCHS, 
    validation_data=val_dataset,
    class_weight=CLASS_WEIGHTS)

# evaluate against test set
labels, predictions = [], []
for Xtest, Ytest in test_dataset:
    Ytest_ = model.predict_on_batch(Xtest)
    ytest = np.argmax(Ytest, axis=1)
    ytest_ = np.argmax(Ytest_, axis=1)
    labels.extend(ytest.tolist())
    predictions.extend(ytest.tolist())

print("test accuracy: {:.3f}".format(accuracy_score(labels, predictions)))
print("confusion matrix")
print(confusion_matrix(labels, predictions)) # 결과 wow.....

[==================================================] 100.0% 376.1/376.1MB downloaded
Embedding matrix: (9010, 300)
Model: "spam_classifier_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  2703000   
                                                                 
 spatial_dropout1d (SpatialD  multiple                 0         
 ropout1D)                                                       
                                                                 
 conv1d (Conv1D)             multiple                  230656    
                                                                 
 global_max_pooling1d (Globa  multiple                 0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               multiple                  514    